In [ ]:
# Just trying out neural style transfer stuff 😅

# First let's install the library that helps us
!pip install torch torchvision

# importing some useful libs
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt
import copy

# resize images to something smallish
imsize = 256

# transforms to convert images to tensor and normalize them
loader = transforms.Compose([
    transforms.Resize((imsize, imsize)),
    transforms.ToTensor()
])

# this function loads and prepares the image
def image_loader(image_path):
    image = Image.open(image_path)
    image = loader(image).unsqueeze(0)  # add batch dimension
    return image.to("cuda" if torch.cuda.is_available() else "cpu", torch.float)

# now load your content and style images
# you can upload your own images in colab using the upload button

from google.colab import files
uploaded = files.upload()

# replace these with your actual file names
content_img = image_loader("your_content_image.jpg")
style_img = image_loader("your_style_image.jpg")

# quick function to show images
def imshow(tensor, title=None):
    image = tensor.cpu().clone()
    image = image.squeeze(0)
    image = transforms.ToPILImage()(image)
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()

# display both images to see
imshow(content_img, title='Content Image')
imshow(style_img, title='Style Image')

# use VGG19 model from torchvision
cnn = models.vgg19(pretrained=True).features.to("cuda" if torch.cuda.is_available() else "cpu").eval()

# mean and std for normalizing input image
normalization_mean = torch.tensor([0.485, 0.456, 0.406])
normalization_std = torch.tensor([0.229, 0.224, 0.225])

# simple class to normalize
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = mean.clone().detach().view(-1, 1, 1)
        self.std = std.clone().detach().view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

# content loss to compare features
class ContentLoss(nn.Module):
    def __init__(self, target):
        super(ContentLoss, self).__init__()
        self.target = target.detach()
    def forward(self, input):
        self.loss = nn.functional.mse_loss(input, self.target)
        return input

# style loss to compare textures/patterns
def gram_matrix(input):
    a, b, c, d = input.size()
    features = input.view(a * b, c * d)
    G = torch.mm(features, features.t())
    return G.div(a * b * c * d)

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
    def forward(self, input):
        G = gram_matrix(input)
        self.loss = nn.functional.mse_loss(G, self.target)
        return input

# layers where we want to compute style and content losses
content_layers = ['conv_4']
style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

# building the model
def get_style_model_and_losses(cnn, norm_mean, norm_std, style_img, content_img):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(norm_mean, norm_std).to("cuda" if torch.cuda.is_available() else "cpu")
    content_losses = []
    style_losses = []

    model = nn.Sequential(normalization)

    i = 0  # increment when we see a conv layer
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            name = 'layer_{}'.format(i)

        model.add_module(name, layer)

        if name in content_layers:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # trim off the layers after the last content/style loss
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
    model = model[:i+1]

    return model, style_losses, content_losses

# we start from content image
input_img = content_img.clone()

# make it require gradient
input_img.requires_grad_(True)

# get the model and losses
model, style_losses, content_losses = get_style_model_and_losses(
    cnn, normalization_mean, normalization_std, style_img, content_img
)

# optimizer
optimizer = optim.LBFGS([input_img])

# run the style transfer
print("Working on it... might take a while ⏳")
run = [0]
while run[0] <= 300:

    def closure():
        input_img.data.clamp_(0, 1)
        optimizer.zero_grad()
        model(input_img)
        style_score = 0
        content_score = 0

        for sl in style_losses:
            style_score += sl.loss
        for cl in content_losses:
            content_score += cl.loss

        loss = style_score * 1000000 + content_score
        loss.backward()
        run[0] += 1

        if run[0] % 50 == 0:
            print("step {}: Style Loss : {:4f} Content Loss: {:4f}".format(run[0], style_score.item(), content_score.item()))

        return loss

    optimizer.step(closure)

# clamp final image and show it
input_img.data.clamp_(0, 1)
imshow(input_img, title='Stylized Image ✨')
